# PROJECT:-

# BY- ANUPAM SRIVASTAVA

In [1]:
pip install newspaper3k

Note: you may need to restart the kernel to use updated packages.


In [41]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv
from urllib.request import urlopen
import re

# Crawling News from Hindustan Times Website

In [42]:
url = "https://www.ndtv.com/world-news/"
r = requests.get(url)

In [43]:
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.findAll('a', attrs = {'href':re.compile("^https://www.ndtv.com/world-news/")})

In [44]:
news=[]
for row in table: 
    if row['href'] not in news and not row['href']=="https://www.ndtv.com/world-news/":
        news.append(row['href'])  
print(news)        

['https://www.ndtv.com/world-news/north-korean-leader-kim-jong-uns-train-likely-spotted-in-resort-town-amid-health-rumours-report-2218393?News_Trending', 'https://www.ndtv.com/world-news/coronavirus-pandemic-all-covid19-patients-in-wuhan-discharged-says-china-2218484', 'https://www.ndtv.com/world-news/north-korean-leader-kim-jong-uns-train-likely-spotted-in-resort-town-amid-health-rumours-report-2218393', 'https://www.ndtv.com/world-news/us-president-donald-trump-russias-vladimir-putin-issue-rare-joint-statement-on-cooperation-2218385', 'https://www.ndtv.com/world-news/coronavirus-in-us-not-worth-time-effort-says-donald-trump-on-pressers-after-disinfectant-gaffe-2218362', 'https://www.ndtv.com/world-news/covid-19-death-count-global-coronavirus-death-count-crosses-2-lakhs-who-warns-over-immunity-2218355', 'https://www.ndtv.com/world-news/coronavirus-france-reports-369-new-covid-19-deaths-in-last-24-hours-2218327', 'https://www.ndtv.com/world-news/coronvirus-china-eases-restrictions-on-e

In [45]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jupyterlab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [46]:
df=[]
for i in news:
    article = Article(i, language="en")
    article.download()
    article.parse()
    article.nlp()
    data={}
    data['Title']=article.title
    data['Text']=article.text
    data['Summary']=article.summary
    data['Keywords']=article.keywords
    df.append(data)

In [47]:
dataset=pd.DataFrame(df)
dataset.head()

,Title,Text,Summary,Keywords
0,Kim Jong Un's Train Likely Spotted In Resort T...,There have been conflicting reports about Kim ...,There have been conflicting reports about Kim ...,"[korean, train, north, whereabouts, spotted, r..."
1,All Coronavirus Patients In Wuhan Have Been Di...,Covid -19 Cases: Wuhan in China had reported 4...,Covid -19 Cases: Wuhan in China had reported 4...,"[wuhan, cases, 56, staff, discharged, coronavi..."
2,Kim Jong Un's Train Likely Spotted In Resort T...,There have been conflicting reports about Kim ...,There have been conflicting reports about Kim ...,"[korean, train, north, whereabouts, spotted, r..."
3,"Donald Trump, Vladimir Putin Issue Rare Joint ...",Donald Trump had hoped to travel to Moscow to ...,Donald Trump had hoped to travel to Moscow to ...,"[statement, donald, elbe, river, cooperation, ..."
4,"""Not Worth Time And Effort"": Donald Trump On P...",He appeared to confirm media reports that he w...,He appeared to confirm media reports that he w...,"[house, donald, effort, coronavirus, worth, tr..."


# Model for predicting virality of news

In [48]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import csv

In [49]:
FILEPATH="/resources/NewsVirality/OnlineNewsPopularity.csv"

In [50]:
import os
os.getcwd()

'/resources/NewsVirality'

In [51]:
def clean_cols(data):
    """Clean the column names by stripping and lowercase."""
    clean_col_map = {x: x.lower().strip() for x in list(data)}
    return data.rename(index=str, columns=clean_col_map)

def TrainTestSplit(X, Y, R=0, test_size=0.2):
    """Easy Train Test Split call."""
    return train_test_split(X, Y, test_size=test_size, random_state=R)

In [52]:
df = pd.read_csv('/resources/NewsVirality/OnlineNewsPopularity/OnlineNewsPopularity.csv',sep='\s*,\s*')
full_data = clean_cols(df)
full_data.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [53]:
train_set, test_set = train_test_split(full_data, test_size=0.20, random_state=42)

x_train = train_set.drop(['url','shares', 'timedelta', 'lda_00','lda_01','lda_02','lda_03','lda_04','num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','rate_positive_words','rate_negative_words','abs_title_subjectivity','abs_title_sentiment_polarity'], axis=1)
y_train = train_set['shares']

x_test = test_set.drop(['url','shares', 'timedelta', 'num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','rate_positive_words','rate_negative_words','abs_title_subjectivity','abs_title_sentiment_polarity'], axis=1)
y_test = test_set['shares']


In [54]:
clf = RandomForestRegressor(random_state=42)
clf.fit(x_train, y_train)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [55]:
rf_res = pd.DataFrame(clf.predict(x_train),list(y_train))

In [56]:
rf_res.reset_index(level=0, inplace=True)
rf_res_df = rf_res.rename(index=str, columns={"index": "Actual shares", 0: "Predicted shares"})
rf_res_df.head()

,Actual shares,Predicted shares
0,16100,11492.9
1,508,1334.8
2,1300,1280.0
3,3100,2940.0
4,6900,5960.0


# Converting Crawled News according to Training Set in UCI Dataset

In [57]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [58]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

In [59]:
def rate_unique(words):
    words=tokenize(words)
    no_order = list(set(words))
    rate_unique=len(no_order)/len(words)
    return rate_unique

In [60]:
def rate_nonstop(words):
    words=tokenize(words)
    filtered_sentence = [w for w in words if not w in stopwords]
    rate_nonstop=len(filtered_sentence)/len(words)
    no_order = list(set(filtered_sentence))
    rate_unique_nonstop=len(no_order)/len(words)
    return rate_nonstop,rate_unique_nonstop

In [61]:
def avg_token(words):
    words=tokenize(words)
    length=[]
    for i in words:
        length.append(len(i))
    return np.average(length)

In [23]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install datefinder

Note: you may need to restart the kernel to use updated packages.


In [62]:
from textblob import TextBlob
import datefinder
import datetime  
from datetime import date 
def day(article_text):
    article=article_text
    if len(list(datefinder.find_dates(article)))>0:
        date=str(list(datefinder.find_dates(article))[0])
        date=date.split()
        date=date[0]
        year, month, day = date.split('-')
        if(int(month)>12 or int(month)<0):
            return "Monday"
        day_name = datetime.date(int(year), int(month), int(day))
        return day_name.strftime("%A")
    return "Monday"

In [63]:
def tokenize(text):
    text=text
    return word_tokenize(text)

In [64]:
pos_words=[]
neg_words=[]
def polar(words):
    all_tokens=tokenize(words)
    for i in all_tokens:
        analysis=TextBlob(i)
        polarity=analysis.sentiment.polarity
        if polarity>0:
            pos_words.append(i)
        if polarity<0:
            neg_words.append(i)
    return pos_words,neg_words

In [65]:
def rates(words):
    words=polar(words)
    pos=words[0]
    neg=words[1]
    all_words=words
    global_rate_positive_words=(len(pos)/len(all_words))/100
    global_rate_negative_words=(len(neg)/len(all_words))/100
    pol_pos=[]
    pol_neg=[]
    for i in pos:
        analysis=TextBlob(i)
        pol_pos.append(analysis.sentiment.polarity)
        avg_positive_polarity=analysis.sentiment.polarity
    for j in neg:
        analysis2=TextBlob(j)
        pol_neg.append(analysis2.sentiment.polarity)
        avg_negative_polarity=analysis2.sentiment.polarity
    min_positive_polarity=min(pol_pos)
    max_positive_polarity=max(pol_pos)
    min_negative_polarity=min(pol_neg)
    max_negative_polarity=max(pol_neg)
    avg_positive_polarity=np.average(pol_pos)
    avg_negative_polarity=np.average(pol_neg)
    return global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity

In [66]:
df2=[]
for i in news:
    print("News - ",i)
    pred_info={}
    article = Article(i, language="en") # en for English 
    article.download()
    article.parse()
    analysis=TextBlob(article.text)
    polarity=analysis.sentiment.polarity
    title_analysis=TextBlob(article.title)
    pred_info['text']=article.text
    pred_info['n_tokens_title']=len(tokenize(article.title))
    pred_info['n_tokens_content']=len(tokenize(article.text))
    pred_info['n_unique_tokens']=rate_unique(article.text)
    pred_info['n_non_stop_words']=rate_nonstop(article.text)[0]
    pred_info['n_non_stop_unique_tokens']=rate_nonstop(article.text)[1]
    pred_info['num_hrefs']=article.html.count("https://www.ndtv.com/world-news/")
    pred_info['num_imgs']=len(article.images)
    pred_info['num_videos']=len(article.movies)
    pred_info['average_token_length']=avg_token(article.text)
    pred_info['num_keywords']=len(article.keywords)
    
    if "life-style" in article.url:
        pred_info['data_channel_is_lifestyle']=1
    else:
        pred_info['data_channel_is_lifestyle']=0
    if "etimes" in article.url:
        pred_info['data_channel_is_entertainment']=1
    else:
        pred_info['data_channel_is_entertainment']=0
    if "business" in article.url:
        pred_info['data_channel_is_bus']=1
    else:
        pred_info['data_channel_is_bus']=0
    if "social media" or "facebook" or "whatsapp" in article.text.lower():
        data_channel_is_socmed=1
        data_channel_is_tech=0
        data_channel_is_world=0
    else:
        data_channel_is_socmed=0
    if ("technology" or "tech" in article.text.lower()) or ("technology" or "tech" in article.url):
        data_channel_is_tech=1
        data_channel_is_socmed=0
        data_channel_is_world=0
    else:
        data_channel_is_tech=0
    if "world" in article.url:
        data_channel_is_world=1
        data_channel_is_tech=0
        data_channel_is_socmed=0
    else:
        data_channel_is_world=0
        
    pred_info['data_channel_is_socmed']=data_channel_is_socmed
    pred_info['data_channel_is_tech']=data_channel_is_tech
    pred_info['data_channel_is_world']=data_channel_is_world
    
    try:
        if day(i)=="Monday":
            pred_info['weekday_is_monday']=1
        else:
            pred_info['weekday_is_monday']=0
        if day(i)=="Tuesday":
            pred_info['weekday_is_tuesday']=1
        else:
            pred_info['weekday_is_tuesday']=0
        if day(i)=="Wednesday":
            pred_info['weekday_is_wednesday']=1
        else:
            pred_info['weekday_is_wednesday']=0
        if day(i)=="Thursday":
            pred_info['weekday_is_thursday']=1
        else:
            pred_info['weekday_is_thursday']=0
        if day(i)=="Friday":
            pred_info['weekday_is_friday']=1
        else:
            pred_info['weekday_is_friday']=0
        if day(i)=="Saturday":
            pred_info['weekday_is_saturday']=1
            pred_info['is_weekend']=1
        else:
            pred_info['weekday_is_saturday']=0
        if day(i)=="Sunday":
            pred_info['weekday_is_sunday']=1
            pred_info['is_weekend']=1
        else:
            pred_info['weekday_is_sunday']=0
            pred_info['is_weekend']=0
    except:
        continue
    pred_info['global_subjectivity']=analysis.sentiment.subjectivity
    pred_info['global_sentiment_polarity']=analysis.sentiment.polarity
    pred_info['global_rate_positive_words']=rates(article.text)[0]
    pred_info['global_rate_negative_words']=rates(article.text)[1]
    pred_info['avg_positive_polarity']=rates(article.text)[2]
    pred_info['min_positive_polarity']=rates(article.text)[3]
    pred_info['max_positive_polarity']=rates(article.text)[4]
    pred_info['avg_negative_polarity']=rates(article.text)[5]
    pred_info['min_negative_polarity']=rates(article.text)[6]
    pred_info['max_negative_polarity']=rates(article.text)[7]    
    pred_info['title_subjectivity']=title_analysis.sentiment.subjectivity
    pred_info['title_sentiment_polarity']=title_analysis.sentiment.polarity
    df2.append(pred_info)

News -  https://www.ndtv.com/world-news/north-korean-leader-kim-jong-uns-train-likely-spotted-in-resort-town-amid-health-rumours-report-2218393?News_Trending
News -  https://www.ndtv.com/world-news/coronavirus-pandemic-all-covid19-patients-in-wuhan-discharged-says-china-2218484
News -  https://www.ndtv.com/world-news/north-korean-leader-kim-jong-uns-train-likely-spotted-in-resort-town-amid-health-rumours-report-2218393
News -  https://www.ndtv.com/world-news/us-president-donald-trump-russias-vladimir-putin-issue-rare-joint-statement-on-cooperation-2218385
News -  https://www.ndtv.com/world-news/coronavirus-in-us-not-worth-time-effort-says-donald-trump-on-pressers-after-disinfectant-gaffe-2218362
News -  https://www.ndtv.com/world-news/covid-19-death-count-global-coronavirus-death-count-crosses-2-lakhs-who-warns-over-immunity-2218355
News -  https://www.ndtv.com/world-news/coronavirus-france-reports-369-new-covid-19-deaths-in-last-24-hours-2218327
News -  https://www.ndtv.com/world-news

In [67]:
pred_df=pd.DataFrame(df2)
pred_test=pred_df.drop(['text'],axis=1)
pred_df.head()

,text,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_imgs,num_videos,average_token_length,...,global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity
0,There have been conflicting reports about Kim ...,15,712,0.438202,0.681180,0.370787,15,11,0,4.480337,...,0.045,0.09,0.283532,0.05,0.5,-0.214749,-0.5,-0.0125,1.0,0.00
1,Covid -19 Cases: Wuhan in China had reported 4...,11,152,0.625000,0.697368,0.467105,15,12,0,3.960526,...,0.375,0.36,0.283005,0.05,0.5,-0.214749,-0.5,-0.0125,0.0,0.00
2,There have been conflicting reports about Kim ...,15,712,0.438202,0.681180,0.370787,15,11,0,4.480337,...,0.525,0.45,0.282606,0.05,0.5,-0.214749,-0.5,-0.0125,1.0,0.00
3,Donald Trump had hoped to travel to Moscow to ...,11,430,0.493023,0.665116,0.409302,15,11,0,4.809302,...,0.860,0.73,0.283997,0.05,0.5,-0.230159,-0.6,-0.0125,0.9,0.30
4,He appeared to confirm media reports that he w...,16,444,0.560811,0.655405,0.436937,15,12,0,4.592342,...,1.055,0.79,0.303946,0.05,0.5,-0.239132,-0.6,-0.0125,0.1,-0.15


# Final Results depicting the Likelihood of Virality of News

In [68]:
test2=pd.DataFrame(clf.predict(pred_test),pred_df['text'])
test2.reset_index(level=0, inplace=True)
test2 = test2.rename(index=str, columns={"index": "News", 0: "Virality"})
test2

,text,Virality
0,There have been conflicting reports about Kim ...,2909.8
1,Covid -19 Cases: Wuhan in China had reported 4...,14581.3
2,There have been conflicting reports about Kim ...,4129.8
3,Donald Trump had hoped to travel to Moscow to ...,17232.7
4,He appeared to confirm media reports that he w...,10315.9
5,The World Health Organization warned on Saturd...,6250.0
6,The number of people in intensive is dropping ...,21178.9
7,The new ruling applies to products such as cor...,10975.9
8,The UK's official coronavirus death toll passe...,6768.3
9,She lost consciousness for 3 weeks and was dec...,17680.0


# Hope This will fulfill you requirements. 